In [33]:
import pandas as pd
import pulp

def create_fleet_optimization_model(demand_df, vehicle_options, max_carbon_per_year):
    years = sorted(demand_df['Year'].unique())
    size_distance_combos = demand_df[['Size', 'Distance']].drop_duplicates().values
    
    model = pulp.LpProblem("Fleet_Optimization", pulp.LpMinimize)
    
    # Decision Variables
    purchases = pulp.LpVariable.dicts("purchase", ((y, s, d, v) for y in years for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()), lowBound=0, cat='Integer')
    sales = pulp.LpVariable.dicts("sell", ((y, p, s, d, v) for y in years for p in years if p < y for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()), lowBound=0, cat='Integer')
    
    # Objective Function
    model += (
        pulp.lpSum(purchases[y, s, d, v] * vehicle_options[(s, d)][v]['purchase_cost'] for y in years for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()) +
        pulp.lpSum(demand_df[(demand_df['Year']==y) & (demand_df['Size']==s) & (demand_df['Distance']==d)]['Demand'].iloc[0] * vehicle_options[(s, d)][v]['cost_per_km'] for y in years for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()) -
        pulp.lpSum(sales[y, p, s, d, v] * vehicle_options[(s, d)][v]['resale_value'][y - p] for y in years for p in years if p < y for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys())
    )
    
    # Constraints
    for y in years:
        for s, d in size_distance_combos:
            demand = demand_df[(demand_df['Year']==y) & (demand_df['Size']==s) & (demand_df['Distance']==d)]['Demand'].iloc[0]
            
            # Fleet Availability
            available_vehicles = (
                pulp.lpSum(purchases[p, s, d, v] for p in years if p <= y for v in vehicle_options[(s, d)].keys() if y >= vehicle_options[(s, d)][v]['available_from_year']) -
                pulp.lpSum(sales[y_sale, p, s, d, v] for y_sale in years if y_sale <= y for p in years if p < y_sale for v in vehicle_options[(s, d)].keys())
            )
            model += pulp.lpSum(available_vehicles * vehicle_options[(s, d)][v]['capacity_per_year'] for v in vehicle_options[(s, d)].keys()) >= demand
            
            # Sales Constraint: Cannot sell more than 20% of the fleet
            if y > min(years):  # Ensure there is a previous year
                previous_fleet_size = (
                    pulp.lpSum(purchases[p, s, d, v] for p in years if p < y for v in vehicle_options[(s, d)].keys()) -
                    pulp.lpSum(sales[y_sale, p, s, d, v] for y_sale in years if y_sale < y for p in years if p < y_sale for v in vehicle_options[(s, d)].keys())
                )
                model += pulp.lpSum(sales[y, p, s, d, v] for p in years if p < y for v in vehicle_options[(s, d)].keys()) <= 0.2 * previous_fleet_size
            
        # Carbon Emissions Constraint
        total_carbon = pulp.lpSum(
            demand_df[(demand_df['Year']==y) & (demand_df['Size']==s) & (demand_df['Distance']==d)]['Demand'].iloc[0] * vehicle_options[(s, d)][v]['carbon_per_km']
            for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()
        )
        model += total_carbon <= max_carbon_per_year[y]
    
    # Solve model
    model.solve()
    
    # Extract Results
    results = {
        'purchases': {(y, s, d, v): int(purchases[y, s, d, v].value()) if purchases[y, s, d, v].value() is not None else 0 for y in years for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()},
        'sales': {(y, p, s, d, v): int(sales[y, p, s, d, v].value()) if sales[y, p, s, d, v].value() is not None else 0 for y in years for p in years if p < y for s, d in size_distance_combos for v in vehicle_options[(s, d)].keys()},
        'objective_value': pulp.value(model.objective)
    }
    
    return results


In [37]:
import pandas as pd

# Define vehicle options with different availability, costs, and emissions
vehicle_options = {
    ('S1', 'D1'): {
        'V1': {
            'vehicle_type': 'Type_A',
            'cost_per_km': 0.5,
            'carbon_per_km': 0.3,
            'purchase_cost': 50000,
            'resale_value': {1: 40000, 2: 32000, 3: 25600},
            'capacity_per_year': 100,
            'available_from_year': 2025
        },
        'V2': {
            'vehicle_type': 'Type_B',
            'cost_per_km': 0.1,  # Reduced cost
            'carbon_per_km': 0.1,  # Lower emissions
            'purchase_cost': 4000,  # Lower cost
            'resale_value': {1: 38000, 2: 31000, 3: 24800},
            'capacity_per_year': 500,  # Higher capacity
            'available_from_year': 2026
        }
    },
    ('S2', 'D1'): {
        'V3': {
            'vehicle_type': 'Type_C',
            'cost_per_km': 0.6,
            'carbon_per_km': 0.35,
            'purchase_cost': 60000,
            'resale_value': {1: 48000, 2: 36000, 3: 28800},
            'capacity_per_year': 1000,
            'available_from_year': 2025
        }
    }
}

# Create demand DataFrame
demand_data = {
    'Year': [2025, 2025, 2026, 2026, 2027, 2027],
    'Size': ['S1', 'S2', 'S1', 'S2', 'S1', 'S2'],
    'Distance': ['D1', 'D1', 'D1', 'D1', 'D1', 'D1'],
    'Demand': [1000, 10000, 2000, 20000, 3000, 30000]
}
demand_df = pd.DataFrame(demand_data)

# Create carbon emissions constraints per year
max_carbon_per_year = {2025: 200000, 2026: 190000, 2027: 180000}

# Run the fleet optimization function
results = create_fleet_optimization_model(demand_df, vehicle_options, max_carbon_per_year)

# Print results
print("Optimal Vehicle Purchases:", results['purchases'])
print("Optimal Vehicle Sales:", results['sales'])
print("Total Cost (Objective Function Value):", results['objective_value'])


Optimal Vehicle Purchases: {(np.int64(2025), 'S1', 'D1', 'V1'): 0, (np.int64(2025), 'S1', 'D1', 'V2'): 0, (np.int64(2025), 'S2', 'D1', 'V3'): 0, (np.int64(2026), 'S1', 'D1', 'V1'): 0, (np.int64(2026), 'S1', 'D1', 'V2'): 0, (np.int64(2026), 'S2', 'D1', 'V3'): 0, (np.int64(2027), 'S1', 'D1', 'V1'): 0, (np.int64(2027), 'S1', 'D1', 'V2'): 0, (np.int64(2027), 'S2', 'D1', 'V3'): 0}
Optimal Vehicle Sales: {(np.int64(2026), np.int64(2025), 'S1', 'D1', 'V1'): 0, (np.int64(2026), np.int64(2025), 'S1', 'D1', 'V2'): 0, (np.int64(2026), np.int64(2025), 'S2', 'D1', 'V3'): 0, (np.int64(2027), np.int64(2025), 'S1', 'D1', 'V1'): 0, (np.int64(2027), np.int64(2025), 'S1', 'D1', 'V2'): 0, (np.int64(2027), np.int64(2025), 'S2', 'D1', 'V3'): 0, (np.int64(2027), np.int64(2026), 'S1', 'D1', 'V1'): 0, (np.int64(2027), np.int64(2026), 'S1', 'D1', 'V2'): 0, (np.int64(2027), np.int64(2026), 'S2', 'D1', 'V3'): 0}
Total Cost (Objective Function Value): 39600.0


In [7]:
import pandas as pd

dfs = []
for YEAR in range(2023, 2039):
    df = pd.read_csv(f'topsis_results_{YEAR}.csv', index_col=False)
    top_ranked = df[df["Rank"] == 1].copy()
    
    # Add a column to indicate the year (if needed for reference)
    top_ranked["Year"] = YEAR
    
    # Append the filtered DataFrame to the list
    dfs.append(top_ranked)

# Combine all filtered DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
final_df.to_csv('top_ranked_combined.csv', index=False)

print("CSV files processed and combined successfully!")


CSV files processed and combined successfully!
